Reading Compounds

Featurizing, Element Fraction and MagPie

In [1]:
import re, numpy as np, os, sys, pandas
from pymatgen.core.composition import Composition
import data_utils
from pymatgen.core.composition import Composition
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from matminer.featurizers.conversions import StrToComposition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
data_path = 'Compounds_input.csv'  ## create excel with compounds listed in the firs column
data_DFT = pandas.read_csv(data_path)
data_DFT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pretty_comp  59 non-null     object
 1   material_id  59 non-null     object
 2   nsites       59 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.5+ KB


In [3]:
data_DFT.describe()

,nsites
count,59.000000
mean,16.406780
std,14.441462
min,2.000000
25%,6.000000
50%,11.000000
75%,22.000000
max,56.000000


In [4]:
%%time
data_DFT['comp_obj'] = data_DFT['pretty_comp'].apply(lambda x: Composition(x))

CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
%%time
data_DFT['composition'] = data_DFT['comp_obj'].apply(lambda x: x.reduced_formula)

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
data_DFT['nelems'] = data_DFT['comp_obj'].apply(lambda x: len(x))

In [7]:
data_DFT.groupby('nelems').count()

,pretty_comp,material_id,nsites,comp_obj,composition
nelems,,,,,
2,59,59,59,59,59


In [8]:
data_DFT.query('nelems > 1', inplace=True)
print (data_DFT.groupby('nelems').count())
data_DFT.shape

        pretty_comp  material_id  nsites  comp_obj  composition
nelems                                                         
2                59           59      59        59           59


(59, 6)

In [9]:
%%time
data_DFT['comp_dict'] = data_DFT['composition'].apply(lambda x: data_utils.parse_formula(x))

CPU times: total: 0 ns
Wall time: 180 µs


In [10]:
%%time
elements_present = set()
for i, row in data_DFT.iterrows():
    elements_present |= set(row['comp_dict'].keys())
print (elements_present)

{'C', 'N', 'U', 'O', 'F', 'Si', 'Mo'}
CPU times: total: 0 ns
Wall time: 1 ms


In [11]:
elements = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
            'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
            'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 
            'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
            'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa',
            'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
            'Ds', 'Rg', 'Cn']


elements_tl = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

elem_pos = dict()
i=0
for el in elements:
   elem_pos[el] = i
   i+=1

In [12]:
print (elements_present, len(elements_present))
print ([e for e in elements_present if e not in elements_tl])
print ([e for e in elements_tl if e not in elements_present])

{'C', 'N', 'U', 'O', 'F', 'Si', 'Mo'} 7
[]
['H', 'Li', 'Be', 'B', 'Na', 'Mg', 'Al', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'Np', 'Pu']


In [13]:
data_DFT['comp_fractions'] = data_DFT['comp_dict'].apply(lambda x: data_utils.get_fractions(x))

In [14]:
print (data_DFT[:1])
#oqmd_data['solution_term'] = oqmd_data['comp_obj'].apply(lambda x: compute_mixing_term(x))
print (data_DFT.shape)
data_DFT = data_DFT[~data_DFT['comp_fractions'].isnull()]
print (data_DFT.shape)

  pretty_comp material_id  nsites comp_obj composition  nelems  \
0       U11O5   mp-673671      16   (U, O)       U11O5       2   

                    comp_dict  \
0  {'U': 0.6875, 'O': 0.3125}   

                                      comp_fractions  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3125, 0.0, 0....  
(59, 8)
(59, 8)


In [15]:
for i,e in enumerate(elements_tl):
    data_DFT[e] = [ x[i] for x in data_DFT['comp_fractions']]
data_DFT.shape

(59, 94)

In [16]:
data_DFT.columns

Index(['pretty_comp', 'material_id', 'nsites', 'comp_obj', 'composition',
       'nelems', 'comp_dict', 'comp_fractions', 'H', 'Li', 'Be', 'B', 'C', 'N',
       'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti',
       'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
       'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd',
       'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce',
       'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
       'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb',
       'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu'],
      dtype='object')

In [17]:
feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])

In [18]:
feature_labels = feature_calculators.feature_labels()

In [19]:
print (feature_labels, len(feature_labels))

['0-norm', '2-norm', '3-norm', '5-norm', '7-norm', '10-norm', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber', 'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber', 'MagpieData avg_dev MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData minimum AtomicWeight', 'MagpieData maximum AtomicWeight', 'MagpieData range AtomicWeight', 'MagpieData mean AtomicWeight', 'MagpieData avg_dev AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT', 'MagpieData range MeltingT', 'MagpieData mean MeltingT', 'MagpieData avg_dev MeltingT', 'MagpieData mode MeltingT', 'MagpieData minimum Column', 'MagpieData maximum Column', 'MagpieData range Column', 'MagpieData mean Column', 'MagpieData avg_dev Column', 'MagpieData mode Column', 'Magpi

In [20]:
%%time
data_DFT = feature_calculators.featurize_dataframe(data_DFT, col_id='comp_obj');

MultipleFeaturizer:   0%|          | 0/59 [00:00<?, ?it/s]

CPU times: total: 78.1 ms
Wall time: 3.38 s


In [21]:
data_DFT[feature_labels].shape

(59, 145)

In [22]:
data_DFT = data_DFT[~data_DFT[feature_labels].isnull().any(axis=1)]
print (data_DFT.shape)

(59, 239)


In [23]:
data_DFT.describe()

,nsites,nelems,H,Li,Be,B,C,N,O,F,...,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,max ionic char,avg ionic char
count,59.000000,59.0,59.0,59.0,59.0,59.0,59.000000,59.000000,59.000000,59.000000,...,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,16.406780,2.0,0.0,0.0,0.0,0.0,0.026836,0.024742,0.443316,0.089387,...,78.101695,54.111544,31.562049,38.711864,1.966808,2.418150,0.513460,1.042498,0.553263,0.104254
std,14.441462,0.0,0.0,0.0,0.0,0.0,0.125183,0.114334,0.366590,0.248779,...,46.441916,48.350098,22.615943,53.803298,0.118302,1.186384,0.672581,0.547679,0.231287,0.046288
min,2.000000,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,48.000000,16.250000,7.791667,12.000000,1.500000,0.000000,0.083333,0.250000,0.065366,0.012256
25%,6.000000,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,51.000000,24.259615,18.106509,12.000000,2.000000,1.566667,0.240385,0.721154,0.497872,0.077608
50%,11.000000,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.666667,0.000000,...,51.000000,26.571429,20.231405,12.000000,2.000000,2.909091,0.272727,0.818182,0.653856,0.118470
75%,22.000000,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.750000,0.000000,...,131.000000,89.750000,42.718750,63.000000,2.000000,3.076923,0.395122,1.185366,0.653856,0.132135
max,56.000000,2.0,0.0,0.0,0.0,0.0,0.666667,0.609756,0.916667,0.857143,...,166.000000,172.333333,83.000000,227.000000,2.000000,4.285714,3.000000,2.625000,0.815480,0.203870


In [24]:
properties = set(data_DFT.columns.tolist()) - (set(feature_labels) | set(elements_present))
print (properties)

{'Dy', 'Ac', 'Cd', 'Y', 'Sb', 'Ca', 'Br', 'Pu', 'Ru', 'Sr', 'H', 'Pm', 'P', 'nsites', 'Te', 'Ti', 'Nb', 'comp_fractions', 'Sm', 'Pd', 'Lu', 'Ho', 'Rb', 'Np', 'Er', 'Ag', 'Ba', 'Ga', 'Rh', 'Ni', 'Li', 'Hg', 'Tb', 'pretty_comp', 'Zr', 'In', 'Sn', 'Bi', 'Sc', 'composition', 'As', 'V', 'Ir', 'Tl', 'Au', 'Fe', 'Zn', 'Cu', 'Ce', 'W', 'S', 'K', 'Al', 'Re', 'Cr', 'Pt', 'Nd', 'Hf', 'Be', 'Pa', 'comp_dict', 'Pb', 'I', 'Pr', 'Mg', 'Co', 'Th', 'B', 'Tc', 'Mn', 'La', 'material_id', 'Os', 'Yb', 'Cs', 'Gd', 'Kr', 'comp_obj', 'Ge', 'nelems', 'Ta', 'Eu', 'Se', 'Tm', 'Na', 'Cl', 'Xe'}


In [25]:
for col in ['comp_fractions', 'comp_dict', 'comp_obj']: data_DFT.drop(col, axis=1, inplace=True)
data_DFT.shape

(59, 236)

In [26]:
properties = set(data_DFT.columns.tolist()) - (set(feature_labels) | set(elements_present) |{'pretty_comp'})
print (properties)

{'Dy', 'Ac', 'Cd', 'Y', 'Sb', 'Ca', 'Br', 'Pu', 'Ru', 'Sr', 'H', 'Pm', 'P', 'nsites', 'Te', 'Ti', 'Nb', 'Sm', 'Pd', 'Lu', 'Ho', 'Rb', 'Np', 'Er', 'Ag', 'Ba', 'Ga', 'Rh', 'Ni', 'Li', 'Hg', 'Tb', 'Zr', 'In', 'Sn', 'Bi', 'Sc', 'composition', 'As', 'V', 'Ir', 'Tl', 'Au', 'Fe', 'Zn', 'Cu', 'Ce', 'W', 'S', 'K', 'Al', 'Re', 'Cr', 'Pt', 'Nd', 'Hf', 'Be', 'Pa', 'Pb', 'I', 'Pr', 'Mg', 'Co', 'Th', 'B', 'Tc', 'Mn', 'La', 'material_id', 'Os', 'Yb', 'Cs', 'Gd', 'Kr', 'Ge', 'nelems', 'Ta', 'Eu', 'Se', 'Tm', 'Na', 'Cl', 'Xe'}


In [27]:
data_DFT = data_DFT[['pretty_comp']+elements_tl+list(feature_labels)+list(properties)]

In [28]:
print (data_DFT.columns)
print (data_DFT.shape)

Index(['pretty_comp', 'H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na',
       ...
       'Kr', 'Ge', 'nelems', 'Ta', 'Eu', 'Se', 'Tm', 'Na', 'Cl', 'Xe'],
      dtype='object', length=315)
(59, 315)


In [ ]:
data_DFT.to_csv('Compounds_output.csv', index=False)

In [ ]:
##### to calculate the statistic for the coordenation number using some MP structure. Execute this in loop for all the dataset

import numpy as np
import pandas as pd
from mp_api.client import MPRester
from pymatgen.analysis.local_env import CrystalNN

# Assuming df is already loaded with material IDs in a column named 'material_id'

# Connect to Materials Project API
with MPRester("YOUR KEY HERE") as mpr:
    
    # Initialize the CrystalNN object
    cnn = CrystalNN()

    # Lists to store the results
    min_cn_list = []
    max_cn_list = []
    avg_cn_list = []

    # Iterate over the material_id column in the existing DataFrame
    for material_id in data_DFT['material_id']:
        # Get the structure for each material_id
        struc = mpr.get_structure_by_material_id(material_id)
        
        # List to store coordination numbers for the current structure
        cn_list = []

        # Iterate over all sites in the structure and get coordination numbers
        for i in range(len(struc)):
            cn = cnn.get_cn(struc, i)
            cn_list.append(cn)

        # Calculate min, max, and average coordination numbers
        min_cn = min(cn_list)
        max_cn = max(cn_list)
        avg_cn = np.mean(cn_list)

        # Append results to the respective lists
        min_cn_list.append(min_cn)
        max_cn_list.append(max_cn)
        avg_cn_list.append(avg_cn)

    # Add the results to the DataFrame
    data_DFT['min_coordination_number'] = min_cn_list
    data_DFT['max_coordination_number'] = max_cn_list
    data_DFT['avg_coordination_number'] = avg_cn_list

    # Save the results to a CSV file
    data_DFT.to_csv("coordination_numbers_valuers.csv", index=False)

    print("Results saved to coordination_numbers.csv")

####add the valuer to the Compound_output file together with space group number

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Minimum Coordination Number: 4
Maximum Coordination Number: 8
Average Coordination Number: 4.903225806451613
